In [ ]:
import os

# If we're running in Colab, we need to clone the non-notebook source from git.
if os.getenv("COLAB_RELEASE_TAG") and not os.path.isdir(
    "/content/mechinterp-experiments"
):
    ip = get_ipython()  #  pyright: ignore[reportUndefinedVariable]
    ip.run_cell_magic(
        "bash",
        "",
        """
    git clone --filter=blob:none --no-checkout https://github.com/evan-lloyd/mechinterp-experiments.git
    cd mechinterp-experiments
    git sparse-checkout init --no-cone
    echo "/next_layer_sae" > .git/info/sparse-checkout
    git checkout next-layer-sae
  """,
    )
    ip.run_line_magic("cd", "mechinterp-experiments/next_layer_sae")
else:
    print("Already cloned source, or not running in Colab.")

Already cloned source, or not running in Colab.


In [1]:
from next_layer_sae.sae import SAE